In [329]:
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_laobai.csv')
data_rfe=np.array(data_rfe)
fea=data_rfe[:,:49]
target=data_rfe[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

# SVM-RFE_RELIEF

In [330]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,alpha,folds):
        #kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            if len(candi_fea)==optim_featur_num:
                return candi_fea
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(i)
            i-=1

In [331]:
rfe_relief_model=rfe_relief_SVM(x_train,y_train,2.2,0.06)

In [332]:
aver_mean_lis1,F_measure_lis1=rfe_relief_model.featu_score(0.15,5)
print(aver_mean_lis1,F_measure_lis1)

[0.797341980270369, 0.7942872801294432, 0.8024544600448875, 0.8024635941333056, 0.7733180228613185, 0.753397880891487, 0.7334907876193957, 0.7350200949945196, 0.7340009917010282, 0.7350240096038415, 0.737066130800146, 0.7192129025523253, 0.7202398350644605, 0.7238112636358892, 0.7105368234250221, 0.6753301320528211, 0.6676783756981053, 0.6620556918419541, 0.6646093219896654, 0.6656375593715748, 0.6523605094211598, 0.6559319379925883, 0.6549128346990971, 0.6538924265358317, 0.6589931624823843, 0.6544000208779164, 0.6498107938827705, 0.6365507072394175, 0.6452228717573986, 0.644715277415314, 0.64471397254554, 0.6380761000052195, 0.6396106268594395, 0.6396106268594395, 0.6396106268594395, 0.6375672007933608, 0.6350161803851975, 0.6365494023696435, 0.6370596064512762, 0.6339970770917063, 0.6324651599770343, 0.6288924265358318, 0.6314447518137689, 0.6273618142909337, 0.6222610783443813, 0.62124197505089, 0.619711362805992, 0.6207343807088053, 0.619715277415314] [0.825448349307543, 0.8175149

In [333]:
aver_mean_lis2,F_measure_lis2=rfe_relief_model.featu_score(0.25,5)
print(aver_mean_lis2,F_measure_lis2)

[0.797341980270369, 0.7942872801294432, 0.8024544600448875, 0.8024635941333056, 0.7733180228613185, 0.753397880891487, 0.7334907876193957, 0.7350200949945196, 0.7340009917010282, 0.7350240096038415, 0.737066130800146, 0.7192129025523253, 0.7202398350644605, 0.7238112636358892, 0.7105368234250221, 0.6753301320528211, 0.6676783756981053, 0.6620556918419541, 0.6646093219896654, 0.6656375593715748, 0.6523605094211598, 0.6559319379925883, 0.6549128346990971, 0.6538924265358317, 0.6589931624823843, 0.6544000208779164, 0.6498107938827705, 0.6365507072394175, 0.6452228717573986, 0.644715277415314, 0.64471397254554, 0.6380761000052195, 0.6396106268594395, 0.6396106268594395, 0.6396106268594395, 0.6375672007933608, 0.6350161803851975, 0.6365494023696435, 0.6370596064512762, 0.6339970770917063, 0.6324651599770343, 0.6288924265358318, 0.6314447518137689, 0.6273618142909337, 0.6222610783443813, 0.62124197505089, 0.619711362805992, 0.6207343807088053, 0.619715277415314] [0.825448349307543, 0.8175149

In [334]:
aver_mean_lis3,F_measure_lis3=rfe_relief_model.featu_score(0.35,5)
print(aver_mean_lis3,F_measure_lis3)

[0.797341980270369, 0.7942872801294432, 0.8024544600448875, 0.8024635941333056, 0.7733180228613185, 0.753397880891487, 0.7334907876193957, 0.7350200949945196, 0.7340009917010282, 0.7350240096038415, 0.737066130800146, 0.7192129025523253, 0.7202398350644605, 0.7238112636358892, 0.7105368234250221, 0.6753301320528211, 0.6676783756981053, 0.6620556918419541, 0.6646093219896654, 0.6656375593715748, 0.6523605094211598, 0.6559319379925883, 0.6549128346990971, 0.6538924265358317, 0.6589931624823843, 0.6544000208779164, 0.6498107938827705, 0.6365507072394175, 0.6452228717573986, 0.644715277415314, 0.64471397254554, 0.6380761000052195, 0.6396106268594395, 0.6396106268594395, 0.6396106268594395, 0.6375672007933608, 0.6350161803851975, 0.6365494023696435, 0.6370596064512762, 0.6339970770917063, 0.6324651599770343, 0.6288924265358318, 0.6314447518137689, 0.6273618142909337, 0.6222610783443813, 0.62124197505089, 0.619711362805992, 0.6207343807088053, 0.619715277415314] [0.825448349307543, 0.8175149

In [335]:
aver_mean_lis4,F_measure_lis4=rfe_relief_model.featu_score(0.45,5)
print(aver_mean_lis4,F_measure_lis4)

[0.797341980270369, 0.7942872801294432, 0.8024544600448875, 0.8024635941333056, 0.7733180228613185, 0.753397880891487, 0.7334907876193957, 0.7350200949945196, 0.7340009917010282, 0.7350240096038415, 0.737066130800146, 0.7192129025523253, 0.7202398350644605, 0.7238112636358892, 0.7105368234250221, 0.6753301320528211, 0.6676783756981053, 0.6620556918419541, 0.6646093219896654, 0.6656375593715748, 0.6523605094211598, 0.6559319379925883, 0.6549128346990971, 0.6538924265358317, 0.6589931624823843, 0.6544000208779164, 0.6498107938827705, 0.6365507072394175, 0.6452228717573986, 0.644715277415314, 0.64471397254554, 0.6380761000052195, 0.6396106268594395, 0.6396106268594395, 0.6396106268594395, 0.6375672007933608, 0.6350161803851975, 0.6365494023696435, 0.6370596064512762, 0.6339970770917063, 0.6324651599770343, 0.6288924265358318, 0.6314447518137689, 0.6273618142909337, 0.6222610783443813, 0.62124197505089, 0.619711362805992, 0.6207343807088053, 0.619715277415314] [0.825448349307543, 0.8175149

In [336]:
aver_mean_lis5,F_measure_lis5=rfe_relief_model.featu_score(0.55,5)
print(aver_mean_lis5,F_measure_lis5)

[0.797341980270369, 0.7942872801294432, 0.8024544600448875, 0.8024635941333056, 0.7733180228613185, 0.753397880891487, 0.7334907876193957, 0.7350200949945196, 0.7340009917010282, 0.7350240096038415, 0.737066130800146, 0.7350266193433896, 0.7421773057048906, 0.7238112636358892, 0.7105368234250221, 0.6753301320528211, 0.6676783756981053, 0.6620556918419541, 0.6646093219896654, 0.6656375593715748, 0.6523605094211598, 0.6559319379925883, 0.6549128346990971, 0.6538924265358317, 0.6589931624823843, 0.6544000208779164, 0.6498107938827705, 0.6365507072394175, 0.6452228717573986, 0.644715277415314, 0.64471397254554, 0.6380761000052195, 0.6396106268594395, 0.6396106268594395, 0.6396106268594395, 0.6375672007933608, 0.6350161803851975, 0.6365494023696435, 0.6370596064512762, 0.6339970770917063, 0.6324651599770343, 0.6288924265358318, 0.6314447518137689, 0.6273618142909337, 0.6222610783443813, 0.62124197505089, 0.619711362805992, 0.6207343807088053, 0.619715277415314] [0.825448349307543, 0.8175149

In [337]:
aver_mean_lis6,F_measure_lis6=rfe_relief_model.featu_score(0.65,5)
print(aver_mean_lis6,F_measure_lis6)

[0.797341980270369, 0.7942872801294432, 0.8024544600448875, 0.8024635941333056, 0.7733180228613185, 0.753397880891487, 0.7334907876193957, 0.7350200949945196, 0.7340009917010282, 0.7350240096038415, 0.737066130800146, 0.7350266193433896, 0.7421773057048906, 0.7238112636358892, 0.7105368234250221, 0.6753301320528211, 0.6676783756981053, 0.6620556918419541, 0.6646093219896654, 0.6656375593715748, 0.6523605094211598, 0.6559319379925883, 0.6549128346990971, 0.6538924265358317, 0.6589931624823843, 0.6544000208779164, 0.6498107938827705, 0.6365507072394175, 0.6452228717573986, 0.644715277415314, 0.64471397254554, 0.6380761000052195, 0.6396106268594395, 0.6396106268594395, 0.6396106268594395, 0.6375672007933608, 0.6350161803851975, 0.6365494023696435, 0.6370596064512762, 0.6339970770917063, 0.6324651599770343, 0.6288924265358318, 0.6314447518137689, 0.6273618142909337, 0.6222610783443813, 0.62124197505089, 0.619711362805992, 0.6207343807088053, 0.619715277415314] [0.825448349307543, 0.8175149

In [338]:
aver_mean_lis7,F_measure_lis7=rfe_relief_model.featu_score(0.75,5)
print(aver_mean_lis7,F_measure_lis7)

[0.797341980270369, 0.7942872801294432, 0.8024544600448875, 0.8024635941333056, 0.7733180228613185, 0.753397880891487, 0.7334907876193957, 0.7350200949945196, 0.7340009917010282, 0.7350240096038415, 0.737066130800146, 0.7350266193433896, 0.7421773057048906, 0.7238112636358892, 0.7105368234250221, 0.6753301320528211, 0.6676783756981053, 0.6620556918419541, 0.6646093219896654, 0.6656375593715748, 0.6523605094211598, 0.6559319379925883, 0.6549128346990971, 0.6538924265358317, 0.6589931624823843, 0.6544000208779164, 0.6498107938827705, 0.6365507072394175, 0.6452228717573986, 0.644715277415314, 0.64471397254554, 0.6380761000052195, 0.6396093219896655, 0.6396106268594395, 0.6396106268594395, 0.6375672007933608, 0.6350161803851975, 0.6365494023696435, 0.6370596064512762, 0.6339970770917063, 0.6324651599770343, 0.6288924265358318, 0.6314447518137689, 0.6273618142909337, 0.6222610783443813, 0.62124197505089, 0.619711362805992, 0.6207343807088053, 0.619715277415314] [0.825448349307543, 0.8175149

In [339]:
aver_mean_lis8,F_measure_lis8=rfe_relief_model.featu_score(0.85,5)
print(aver_mean_lis8,F_measure_lis8)

[0.797341980270369, 0.7942872801294432, 0.7442155122918732, 0.7375737251422309, 0.7477804165144317, 0.7309384623414583, 0.6906297301529307, 0.6941998538545853, 0.6875672007933608, 0.6891121666057727, 0.6875906884492927, 0.6712550237486299, 0.6702346155853646, 0.6661490683229815, 0.6732906205960646, 0.6702359204551385, 0.6768672686465891, 0.6778863719400804, 0.6814525810324129, 0.6804308679993737, 0.6646067122501175, 0.6646054073803433, 0.6620530821024062, 0.666647528576648, 0.6457369904483532, 0.6472676026932512, 0.6416501383161961, 0.6447113628059921, 0.6462406701811159, 0.6452202620178505, 0.638587608956626, 0.6339957722219323, 0.6339957722219322, 0.6365480974998695, 0.6350174852549715, 0.634507281173339, 0.6329766689284411, 0.6324664648468084, 0.6304256485202778, 0.6294052403570124, 0.6304256485202776, 0.6294052403570124, 0.6283848321937471, 0.6273644240304816, 0.6248134036223185, 0.62124197505089, 0.619711362805992, 0.6207356855785793, 0.619715277415314] [0.825448349307543, 0.81751

In [340]:
aver_mean_lis9,F_measure_lis9=rfe_relief_model.featu_score(0.95,5)
print(aver_mean_lis9,F_measure_lis9)

[0.797341980270369, 0.7942872801294432, 0.7442155122918732, 0.7375737251422309, 0.7477804165144317, 0.7309384623414583, 0.6906297301529307, 0.6941998538545853, 0.6870674356699201, 0.6891121666057727, 0.6875906884492927, 0.6712550237486299, 0.6727882457330758, 0.6804373923482437, 0.6850370583015815, 0.6875867738399709, 0.6865624510673835, 0.6834947022287177, 0.6840062111801244, 0.6697126676757659, 0.6737916905892791, 0.6692011587243594, 0.6686909546427268, 0.6692024635941334, 0.6686922595125007, 0.6712432799206638, 0.6732880108565166, 0.657991022495955, 0.6518646589070412, 0.6472662978234772, 0.6457330758390313, 0.6452267863667206, 0.6431859700401901, 0.6396132365989874, 0.6396119317292135, 0.6365533169789656, 0.6334868730100738, 0.6360378934182369, 0.6304256485202776, 0.632980583537763, 0.6340009917010282, 0.6304308679993736, 0.6263453207369905, 0.6273657289002557, 0.6253236077039512, 0.622772587295788, 0.6217560937418446, 0.6207356855785793, 0.619715277415314] [0.825448349307543, 0.81

In [353]:
np.max(aver_mean_lis1[:12]),np.max(aver_mean_lis2[:12]),np.max(aver_mean_lis3[:12]),np.max(aver_mean_lis4[:12]),np.max(aver_mean_lis5[:12])

(0.8024635941333056,
 0.8024635941333056,
 0.8024635941333056,
 0.8024635941333056,
 0.8024635941333056)

In [354]:
np.max(aver_mean_lis6[:12]),np.max(aver_mean_lis7[:12]),np.max(aver_mean_lis8[:12]),np.max(aver_mean_lis9[:12])

(0.8024635941333056, 0.8024635941333056, 0.797341980270369, 0.797341980270369)

In [355]:
np.argmax(aver_mean_lis1[:12]),np.argmax(aver_mean_lis2[:12]),np.argmax(aver_mean_lis3[:12])

(3, 3, 3)

In [356]:
np.argmax(aver_mean_lis4[:12]),np.argmax(aver_mean_lis5[:12]),np.argmax(aver_mean_lis6[:12])

(3, 3, 3)

In [357]:
np.argmax(aver_mean_lis7[:12]),np.argmax(aver_mean_lis8[:12]),np.argmax(aver_mean_lis9[:12])

(3, 0, 0)

# SVM-RFE

In [347]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,folds):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            if len(candi_fea)==featur_num:
                return candi_fea
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                #print(scor_fea)
                rfe_score[k]=scor_fea
                k=k+1     
            #print(rfe_score)
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(1)

In [348]:
rfe_SVM_model=rfe_SVM(x_train,y_train,2.2,0.06)
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis=rfe_SVM_model.featu_score(5)

In [349]:
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis

([0.797341980270369,
  0.7942872801294432,
  0.8024544600448875,
  0.8024635941333056,
  0.7733180228613185,
  0.753397880891487,
  0.7334907876193957,
  0.7350200949945196,
  0.7340009917010282,
  0.7350240096038415,
  0.737066130800146,
  0.7192129025523253,
  0.7202398350644605,
  0.7238112636358892,
  0.7105368234250221,
  0.6753301320528211,
  0.6676783756981053,
  0.6620556918419541,
  0.6646093219896654,
  0.6656375593715748,
  0.6523605094211598,
  0.6559319379925883,
  0.6549128346990971,
  0.6538924265358317,
  0.6589931624823843,
  0.6544000208779164,
  0.6498107938827705,
  0.6365507072394175,
  0.6452228717573986,
  0.644715277415314,
  0.64471397254554,
  0.6380761000052195,
  0.6396106268594395,
  0.6396106268594395,
  0.6396106268594395,
  0.6375672007933608,
  0.6350161803851975,
  0.6365494023696435,
  0.6370596064512762,
  0.6339970770917063,
  0.6324651599770343,
  0.6288924265358318,
  0.6314447518137689,
  0.6273618142909337,
  0.6222610783443813,
  0.621241975050

# SVM-RELIEF

In [350]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class relief_SVM:
    def __init__(self,x_train,y_train,C,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
        self.Gamma=gamma
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        featu_num=self.X_train.shape[1]
        relief_fea_score=reliefF.reliefF(self.X_train,self.Y_train)
        relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        for i in range(featu_num):
            estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
            candi_fea=relief_candi_fea[:(i+1)]
            aver=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i]=aver
            F_lis[i]=F1 
        return aver_lis,F_lis

In [351]:
relief_SVM_model=relief_SVM(x_train,y_train,2.2,0.06)
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis=relief_SVM_model.featu_score(5)

In [352]:
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis

([0.7953481392557022,
  0.8035218435200168,
  0.8096442924996085,
  0.8137259251526696,
  0.8137285348922176,
  0.8106620909233259,
  0.7175622422882196,
  0.7012174434991388,
  0.7104024218383006,
  0.7042786679889347,
  0.7149942585729944,
  0.7063168745759174,
  0.7104011169685266,
  0.7119291194738766,
  0.7027428362649408,
  0.7047849574612455,
  0.7093833185448093,
  0.7109113210501592,
  0.709890912886894,
  0.6996829166449188,
  0.6966190824155749,
  0.6955986742523097,
  0.6976407954486142,
  0.6971227621483376,
  0.695087165300903,
  0.6976421003183882,
  0.6981509995302468,
  0.6955973693825357,
  0.6986638133514276,
  0.696624301894671,
  0.6986664230909755,
  0.6956025888616317,
  0.6945860953076883,
  0.6930528733232424,
  0.6915209562085703,
  0.6971318962367555,
  0.6930502635836943,
  0.6904953285662091,
  0.6884558171094525,
  0.6894762252727178,
  0.6894723106633959,
  0.6961049637246203,
  0.6930489587139202,
  0.695091079910225,
  0.6940693668771856,
  0.6920285505

In [362]:
laobai_accuracy=[]
laobai_accuracy.append(aver_mean_lis1)
laobai_accuracy.append(aver_mean_lis2)
laobai_accuracy.append(aver_mean_lis3)
laobai_accuracy.append(aver_mean_lis4)
laobai_accuracy.append(aver_mean_lis5)
laobai_accuracy.append(aver_mean_lis6)
laobai_accuracy.append(aver_mean_lis7)
laobai_accuracy.append(aver_mean_lis8)
laobai_accuracy.append(aver_mean_lis9)
laobai_accuracy.append(rfe_SVM_aver_mean_lis)
laobai_accuracy.append(relief_SVM_aver_mean_lis)

In [363]:
colu=[i for i in range(1,50)]
laobai_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(laobai_accuracy))
laobai_acurracy_dataframe.to_csv("laobai_accuracy.csv")

In [364]:
laobai_F_measure=[]
laobai_F_measure.append(F_measure_lis1)
laobai_F_measure.append(F_measure_lis2)
laobai_F_measure.append(F_measure_lis3)
laobai_F_measure.append(F_measure_lis4)
laobai_F_measure.append(F_measure_lis5)
laobai_F_measure.append(F_measure_lis6)
laobai_F_measure.append(F_measure_lis7)
laobai_F_measure.append(F_measure_lis8)
laobai_F_measure.append(F_measure_lis9)
laobai_F_measure.append(rfe_SVM_F_measure_lis)
laobai_F_measure.append(relief_SVM_F_measure_lis)

In [365]:
colu=[i for i in range(1,50)]
laobai_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(laobai_F_measure))
laobai_F_measure_dataframe.to_csv("laobai_F_measure.csv")